In [2]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from tqdm import tqdm

In [3]:
N_arms = 10
N_resources = 5

base = ['F']*N_resources + ['0']*(N_arms-N_resources)
# find all permutations of variants
all_perms = list(itertools.permutations(base))
# remove effective duplicates
variants = []
for perm in tqdm(all_perms):
    # all rotations of a variant are equivalent and so with reflections
    rotations = [perm]
    for i in range(1, N_arms):
        rotations.append(perm[i:] + perm[:i])
    reflections = []
    for rotation in rotations:
        reflections.append(rotation[::-1])
    all_versions = rotations + reflections
    # check if variant is already in list
    all_version_in_list = [x in variants for x in all_versions]
    if not any(all_version_in_list):
        variants.append(perm)

100%|██████████| 3628800/3628800 [01:07<00:00, 53552.48it/s]


In [4]:
variants = [''.join(variant) for variant in variants]
print('Found {} variants'.format(len(variants)))
print('Variants:')
for variant in variants:
    print('\'{}\''.format(variant))

Found 16 variants
Variants:
'FFFFF00000'
'FFFF0F0000'
'FFFF00F000'
'FFF0FF0000'
'FFF0F0F000'
'FFF0F00F00'
'FFF0F000F0'
'FFF00FF000'
'FFF00F0F00'
'FF0FF0F000'
'FF0FF00F00'
'FF0F0FF000'
'FF0F0F0F00'
'FF0F0F00F0'
'FF0F00FF00'
'F0F0F0F0F0'


In [5]:
def variant_to_points(variant,arm_length=1):
    points = []
    for i in range(len(variant)):
        if variant[i] == 'F':
            x = np.cos(i * np.pi / 4) * arm_length
            y = np.sin(i * np.pi / 4) * arm_length
            points.append((x, y)) 
    return points

def get_euclidean_matrix(points):
    matrix = np.zeros((len(points),len(points)))
    for i in range(len(points)):
        for j in range(len(points)):
            matrix[i,j] = np.sqrt((points[i][0] - points[j][0])**2 + (points[i][1] - points[j][1])**2)
    return matrix

def variant_to_spatial_variance(variant):
    points = variant_to_points(variant)
    matrix = get_euclidean_matrix(points)
    return np.mean(matrix[np.triu_indices(len(points),1)])


def variant_to_left_turns(variant):
    # find the indices of the F's
    indices = []
    for i in range(len(variant)):
        if variant[i] == 'F':
            indices.append(i)
    # for each index, find number of left shifts to the other indices assuming circular
    left_turns = []
    for i in range(len(indices)):
        for_i_th = []
        for j in range(len(indices)):
            for_i_th.append((indices[j] - indices[i]) % len(variant))
        left_turns.append(for_i_th)
    return left_turns

def variant_to_right_turns(variant):
    # find the indices of the F's
    indices = []
    for i in range(len(variant)):
        if variant[i] == 'F':
            indices.append(i)
    # for each index, find number of left shifts to the other indices assuming circular
    right_turns = []
    for i in range(len(indices)):
        for_i_th = []
        for j in range(len(indices)):
            for_i_th.append((indices[i] - indices[j]) % len(variant))
        right_turns.append(for_i_th)
    return right_turns

def variant_to_closest_paths(variant):
    left_turns = np.array(variant_to_left_turns(variant))
    right_turns = np.array(variant_to_right_turns(variant))
    # find which turns are the smallest
    left_smaller = left_turns < right_turns
    closer_turns = np.zeros(left_turns.shape)
    closer_turns[left_smaller] = left_turns[left_smaller]
    closer_turns[~left_smaller] = -right_turns[~left_smaller]
    return closer_turns

def variant_to_mean_distance(variant):
    paths = np.abs(variant_to_closest_paths(variant))
    # remove the 0's
    paths = paths[paths > 0]
    # we want to maximize the lower bound of the distance and maximize the upper bound of the distance
    # count how many are more than the mean
    return np.sum(paths > np.mean(paths))/np.sum(paths < np.mean(paths))

def path_to_dice_matrix(path):
    matrix = np.zeros((len(path),len(path)))
    for i in range(len(path)):
        for j in range(len(path)):
            pi = path[i][path[i] != 0]
            pj = path[j][path[j] != 0]
            # count how many elements are in the intersection and calculate the dice coefficient
            matrix[i,j] = len(set(pi).intersection(pj)) / len(set(pi).union(pj))
    return matrix

def variant_to_mean_dice_closest(variant):
    paths = variant_to_closest_paths(variant)
    matrix = path_to_dice_matrix(paths)
    return np.mean(matrix[np.triu_indices(len(paths),1)])

In [6]:
print('Variant','\tSpatial Variance','\tPostural heuristic similarity','\tPostural Separation')
for variant in variants:
    print(variant, "\t{:.5f}".format(variant_to_spatial_variance(variant)), "\t\t{:.5f}".format(variant_to_mean_dice_closest(variant)), "\t\t\t{:.5f}".format(variant_to_mean_distance(variant)))

Variant 	Spatial Variance 	Postural heuristic similarity 	Postural Separation
FFFFF00000 	1.29996 		0.36857 			0.75000
FFFF0F0000 	1.40820 		0.24286 			0.66667
FFFF00F000 	1.40820 		0.20000 			1.00000
FFF0FF0000 	1.46678 		0.20000 			1.00000
FFF0F0F000 	1.48831 		0.24286 			1.00000
FFF0F00F00 	1.40820 		0.20000 			1.50000
FFF0F000F0 	1.23865 		0.25714 			1.00000
FFF00FF000 	1.46678 		0.32000 			1.25000
FFF00F0F00 	1.40820 		0.23429 			1.00000
FF0FF0F000 	1.51644 		0.20000 			1.50000
FF0FF00F00 	1.46678 		0.36857 			2.33333
FF0F0FF000 	1.51644 		0.23429 			1.00000
FF0F0F0F00 	1.48831 		0.24286 			1.00000
FF0F0F00F0 	1.37502 		0.32000 			0.75000
FF0F00FF00 	1.40820 		0.24286 			1.33333
F0F0F0F0F0 	1.44853 		1.00000 			1.00000


In [88]:
# def plot_point(point):
#     plt.figure()
#     plt.plot(point[:,0], point[:,1], 'o')
#     plt.gca().set_aspect('equal', adjustable='box')
#     plt.show()

# for point in points:
#     plot_point(point)


In [89]:
print(variant_to_left_turns('FFFF0000'))
print(variant_to_right_turns('FFFF0000'))
print(variant_to_closest_paths('FFFF0000'))

[[0, 1, 2, 3], [7, 0, 1, 2], [6, 7, 0, 1], [5, 6, 7, 0]]
[[0, 7, 6, 5], [1, 0, 7, 6], [2, 1, 0, 7], [3, 2, 1, 0]]
[[ 0.  1.  2.  3.]
 [-1.  0.  1.  2.]
 [-2. -1.  0.  1.]
 [-3. -2. -1.  0.]]
